# Travaux pratiques 1 

## Année: 2024-2025

## Objectifs
1. Renforcer les concepts à l'aide d'outils comme NumPy et scikit-learn.
2. Appliquer la logique propositionnelle et la tester sur des ensembles de données d'images.
3. Explorer la logique prédicative et la valider sur des ensembles de données d'images.
4. Comprendre les techniques d'analyse de texte, y compris le stemming, la lemmatisation et l'analyse morphologique.

### Exercice 1.0 [★]

Testez le notebook Python Jupyter [rappels](./TP0.ipynb) et familiarisez-vous avec les différentes méthodes des bibliothèques : numpy, scikit-learn, etc.

### Exercice 1.1 [★]

L'exercice consiste à tester la logique propositionnelle. Le fichier CSV (`image_data.csv`) contient des attributs tels que la couleur, la forme, la taille, la texture et la classification après le processus d'analyse d'image. Définissez les propositions ci-dessous et testez les expressions logiques basées sur ces attributs pour évaluer les relations au sein des données.

##### Étape 1 : Charger et Inspecter le Fichier CSV
1. Lire le fichier CSV dans un DataFrame pandas.
2. Afficher les premières lignes du DataFrame pour comprendre la structure.

In [13]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../../data/image_data.csv')

# Inspect the data
print(df.head())

    color      shape  size    texture classification
0   green   triangle   400  polka dot         animal
1     red     square   150  patterned          plant
2   green     square   250  polka dot         object
3  yellow  rectangle   200  polka dot       building
4     red     circle   200  polka dot        vehicle


In [14]:
print(df["texture"].unique())

['polka dot' 'patterned' 'smooth' 'rough' 'striped']


#### Étape 2 : Définir des Propositions Basées sur les Données
1. Créer des propositions booléennes basées sur les colonnes du fichier CSV.
   - `P` : Vérifie si la couleur est bleue.
   - `Q` : Vérifie si la forme est un cercle.
   - `R` : Vérifie si la classification est un animal.
   - `S` : Vérifie si la taille est grande (définir un seuil pour 'grand').
   - `T` : Vérifie si la texture est rugueuse.
   - `U` : Vérifie si la classification est un véhicule.
   - `V` : Vérifie si la classification est un bâtiment.

#### Étape 3 : Définir les Expressions Logiques
Créer des expressions logiques basées sur les propositions définies à l'étape 2.
   - `expr1` : Si la couleur est bleue et la forme est un cercle, alors la classification est un animal.
   - `expr2` : Si la taille est grande et la texture est rugueuse, alors la classification est un véhicule.
   - `expr3` : Si la classification est un bâtiment, que la forme est un cercle, que la couleur est bleue et que la taille est grande, alors la classification est un bâtiment.

#### Étape 4 : Tester les Expressions
1. Tester les expressions logiques sur chaque ligne du DataFrame.
2. Afficher les résultats des expressions pour chaque ligne.

##### Étape 5 : Ajouter une Proposition Composée avec Négation et Disjonction

Créer une nouvelle expression complexe qui teste ce qui suit :
- Si l'objet n'est **pas bleu** ou **a une texture lisse**, alors il est classé comme **non un objet**.

##### Étape 6 : Compter les Lignes Satisfaisantes pour Chaque Expression

Compter le nombre de lignes où chaque expression logique est `True` et comparer les fréquences des propositions satisfaites.

### Exercice 1.2 [★]

##### Étape 1 : Installer et Importer Z3
- Installer la bibliothèque de solveur Z3 (`z3-solver`).
- Importer Z3 et se familiariser avec ses fonctions de base.

In [15]:
%pip install z3-solver

Note: you may need to restart the kernel to use updated packages.


In [1]:
import z3

##### Étape 2 : Définir les Attributs comme Variables de Logique du Premier Ordre
- Définir des variables pour chaque colonne (par exemple, `color`, `shape`, `size`).
- Spécifier les valeurs possibles (par exemple, `color` peut être bleu, rouge, vert, etc.).
- Définir des contraintes pour des attributs tels que **size** étant un entier et d'autres attributs étant des chaînes de caractères.

In [2]:
# Declare Z3 variables
Color = z3.String('color')
Shape = z3.String('shape')
Size = z3.Int('size')
Texture = z3.String('texture')
Classification = z3.String('classification')

#### Étape 3 : Encoder les Propositions Logiques en Logique du Premier Ordre
- Encoder les propositions fournies en utilisant la logique de Z3 (voir l'exercice 1.1).
- Exemple : Si l'objet est bleu et circulaire, alors il est classé comme un animal (`Implies(And(Color == "blue", Shape == "circle"), Classification == "animal")`).

In [3]:
# Define constraints

valid_colors = z3.Or(Color == "blue", Color == "red", Color == "green", Color == "yellow", Color == "purple")
valid_shapes = z3.Or(Shape == "circle", Shape == "square", Shape == "triangle", Shape == "rectangle", Shape == "ellipse")
valid_size = Size >= 100  # Size constraint
valid_textures = z3.Or(Texture == "polka dot", Texture == "smooth", Texture == "patterned", Texture == "rough")
valid_classifications = z3.Or(Classification == "animal", Classification == "plant", Classification == "object", Classification == "vehicle", Classification == "building")

expr5 =z3.Implies(z3.And(Color == "blue", Shape == "circle"), Classification == "animal")
expr6 = z3.Implies(z3.And(Size > 350, Texture == "rough"), Classification == "vehicle")
expr7 = z3.Implies(z3.And(z3.And(Classification == "building", Color == "blue"), z3.And(Shape == "circle", Size >= 350)), Classification == "building")
expr8 = z3.Implies(z3.And(Color == "blue", Shape == "circle"), Classification == "animal")

# Add these constraints to the solver
solver = z3.Solver()
solver.add(valid_colors, valid_shapes, valid_size, valid_textures, valid_classifications, expr5, expr6, expr7, expr8)

##### Étape 4 : Résoudre pour la Satisfaisabilité

- Utiliser le solveur Z3 pour vérifier si les propositions sont satisfaisables.
- Afficher les résultats.

In [4]:
# Check if the solver finds a solution that satisfies the constraints
if solver.check() == z3.sat:
    print("The propositions are satisfiable.")
    model = solver.model()
    print(model)
else:
    print("The propositions are not satisfiable.")

The propositions are satisfiable.
[texture = "polka dot",
 classification = "animal",
 shape = "circle",
 color = "blue",
 size = 100]


##### Étape 5 : Ajouter des Contraintes Supplémentaires
- Ajouter une contrainte qui restreint certaines combinaisons, comme « si l'objet est vert, il ne peut pas être circulaire ».
- Ajouter une autre contrainte où « les objets à pois ne peuvent pas être des véhicules ».

In [5]:
expr9 = z3.Implies(Color == "green", Shape != 'circle')
expr10 = z3.Implies(Texture == "polka dot", Classification != 'vehicle')

solver2 = z3.Solver()
solver2.add(valid_colors, valid_shapes, valid_size, valid_textures, valid_classifications, expr5, expr6, expr7, expr8, expr9, expr10)

if solver2.check() == z3.sat:
    print("The propositions are satisfiable.")
    model2 = solver2.model()
    print(model2)
else:
    print("The propositions are not satisfiable.")

The propositions are satisfiable.
[texture = "polka dot",
 classification = "animal",
 shape = "circle",
 color = "blue",
 size = 100]


## Exercice 1.3 [★★]

Lire le fichier CSV `image_data.csv` et définir les attributs comme des variables Z3 en fonction des données du fichier. Encoder les propositions logiques et les contraintes en utilisant la logique du premier ordre pour chaque ligne, puis résoudre pour la satisfaisabilité.

In [6]:
import pandas as pd
import z3

# Load CSV data
df = pd.read_csv('../../data/image_data.csv')

# Initialize Z3 solver
solver = z3.Solver()

##### Étape 2 : Définir les Variables Z3 Dynamiquement à Partir des Données CSV
Pour chaque ligne du CSV, définir les attributs comme des variables Z3 et s'assurer que les types sont cohérents.

In [8]:
# Define Z3 variables for each attribute dynamically for each row
for index, row in df.iterrows():
    color = z3.String(f'color_{index}')
    shape = z3.String(f'shape_{index}')
    size = z3.Int(f'size_{index}')
    texture = z3.String(f'texture_{index}')
    classification = z3.String(f'classification_{index}')
    
    # Add constraints for valid values
    solver.add(z3.Or(color == row['color'], shape == row['shape'], size == row['size'], texture == row['texture'], classification == row['classification']))


#### Étape 3 : Encoder des Propositions en Logique du Premier Ordre
- Écrire des propositions logiques pour chaque ligne, comme « si un objet est bleu et circulaire, alors il est classé comme un animal ».
- Utiliser l'exercice 1.1 et ajouter des propositions supplémentaires.

In [9]:
# Example FOL for each row
for index, row in df.iterrows():
    expr1 = z3.Implies(z3.And(z3.String(f'color_{index}') == "blue", z3.String(f'shape_{index}') == "circle"), z3.String(f'classification_{index}') == "animal")
    expr2 = z3.Implies(z3.And(z3.Int(f'size_{index}') > 350, z3.String(f'texture_{index}') == "rough"), z3.String(f'classification_{index}') == "vehicle")
    expr3 = z3.Implies(z3.And(z3.And(z3.String(f'classification_{index}') == "building", z3.String(f'color_{index}') == "blue"), z3.And(z3.String(f'shape_{index}') == "circle", z3.Int(f'size_{index}') >= 350)), z3.String(f'classification_{index}') == "building")
    expr4 = z3.Implies(z3.And(z3.String(f'color_{index}') == "blue", z3.String(f'shape_{index}') == "circle"), z3.String(f'classification_{index}') == "animal")
    solver.add(expr1, expr2, expr3, expr4)


#### Étape 4 : Résoudre pour la Satisfaisabilité
Vérifier si les propositions logiques pour les données CSV sont satisfaisables.

In [10]:
# Check if the solver finds a solution that satisfies the constraints
if solver.check() == z3.sat:
    print("The propositions are satisfiable.")
    model = solver.model()
    print(model)
else:
    print("The propositions are not satisfiable.")

The propositions are satisfiable.
[texture_97 = "",
 color_12 = "",
 color_44 = "",
 color_88 = "",
 texture_61 = "",
 color_51 = "purple",
 texture_23 = "",
 shape_83 = "",
 texture_99 = "",
 texture_51 = "",
 classification_51 = "!2f!",
 size_88 = 400,
 classification_18 = "!12!",
 classification_83 = "!50!",
 color_40 = "red",
 texture_84 = "",
 color_23 = "",
 texture_68 = "",
 texture_45 = "",
 texture_74 = "",
 color_92 = "blue",
 color_32 = "",
 texture_9 = "",
 size_26 = 450,
 color_39 = "",
 shape_18 = "",
 classification_41 = "!25!",
 classification_92 = "!59!",
 texture_42 = "",
 texture_82 = "",
 classification_67 = "vehicle",
 classification_1 = "!1!",
 texture_95 = "",
 classification_20 = "!13!",
 texture_57 = "",
 color_93 = "green",
 classification_4 = "!4!",
 shape_36 = "",
 shape_58 = "rectangle",
 classification_59 = "!38!",
 shape_31 = "",
 classification_16 = "!10!",
 color_66 = "",
 classification_33 = "!1d!",
 texture_15 = "smooth",
 color_3 = "",
 classificatio

#### Étape 5 : Ajouter des Contraintes Supplémentaires
- Inclure de nouvelles contraintes (par exemple, « les objets verts ne peuvent pas être circulaires ») et vérifier à nouveau le résultat.

In [13]:
no_green_circle = z3.Implies(z3.String(f'color_{index}') == "green", z3.String(f'shape_{index}') != "circle")
solver.add(no_green_circle)

if solver.check():
    print("The propositions are satisfiable.")
    model = solver.model()
    print(model)

The propositions are satisfiable.
[texture_97 = "",
 color_12 = "yellow",
 color_44 = "yellow",
 color_88 = "",
 texture_61 = "",
 color_51 = "",
 texture_23 = "",
 shape_83 = "",
 texture_99 = "",
 texture_51 = "",
 classification_51 = "vehicle",
 size_88 = 400,
 classification_18 = "!12!",
 classification_83 = "!51!",
 color_40 = "red",
 texture_84 = "",
 color_23 = "green",
 texture_68 = "",
 texture_45 = "",
 texture_74 = "",
 color_32 = "red",
 color_92 = "blue",
 texture_9 = "",
 color_39 = "green",
 shape_18 = "",
 classification_41 = "!28!",
 classification_92 = "!5a!",
 texture_42 = "",
 texture_82 = "",
 classification_67 = "vehicle",
 size_7 = 250,
 classification_1 = "!1!",
 texture_95 = "",
 classification_20 = "!14!",
 texture_57 = "",
 color_93 = "green",
 classification_4 = "!3!",
 shape_36 = "ellipse",
 shape_58 = "",
 classification_59 = "!3a!",
 shape_31 = "!1d!",
 classification_16 = "plant",
 color_66 = "green",
 classification_33 = "!20!",
 texture_15 = "smooth",


#### Étape 6 : Introduire une Contrainte Insatisfaisable
- Ajouter une contrainte conflictuelle qui force un objet à avoir deux attributs mutuellement exclusifs (par exemple, être à la fois bleu et rouge).
- Vérifier la satisfaisabilité et montrer que le modèle devient insatisfaisable.

In [14]:
# Add a conflicting constraint: Object must be both blue and red (which is impossible)
for index in range(len(df)):
    conflicting_constraint = z3.And(z3.String(f'color_{index}') == "blue", z3.String(f'color_{index}') == "red")
    solver.add(conflicting_constraint)

# Check satisfiability after adding the conflicting constraint
if solver.check() == z3.sat:
    print("The propositions are still satisfiable.")
else:
    print("The model is now unsatisfiable due to conflicting constraints.")


The model is now unsatisfiable due to conflicting constraints.


### Exercice 1.4 [★★]

Téléchargez cette page Web de Wikipedia : [https://fr.wikipedia.org/wiki/Paris](https://fr.wikipedia.org/wiki/Paris) et enregistrez le fichier au format HTML. Analysez la page Wikipedia en extrayant et en comptant les mots, les liens, les images, les nombres, les dates, les noms propres et les données structurées à partir des tableaux, tout en différenciant les sections et les paragraphes. Cela implique de télécharger le HTML, de le parser et d'identifier systématiquement le contenu pertinent. Écrivez un programme pour mettre en œuvre ces tâches :

1. **Télécharger le HTML** : Récupérer et enregistrer la page Wikipedia au format HTML.
2. **Charger le Contenu** : Lire et parser le fichier HTML pour analyse.
3. **Analyse des Mots** : Compter les occurrences de mots dans le texte.
4. **Extraction des Liens** : Identifier et catégoriser les liens internes et externes.
5. **Extraction d'Images** : Localiser les images et rassembler leurs URL et tailles.
6. **Extraction de Nombres et de Dates** : Identifier les nombres, les dates et les coordonnées géographiques.
7. **Noms Propres** : Extraire les noms de personnes et de lieux.
8. **Données des Tableaux** : Localiser et extraire les données des tableaux.
9. **Différenciation des Sections** : Identifier les sections et les paragraphes dans le contenu.

#### Analyse de la Page Wikipedia : Paris

Dans ce notebook, des tâches seront effectuées pour extraire et analyser divers éléments de la page Wikipedia de Paris.

##### Étape 1 : Télécharger la Page HTML
Tout d'abord, téléchargez le contenu HTML de la page Wikipedia spécifiée et enregistrez-le en tant que fichier HTML. Nous utilisons la bibliothèque `requests` pour gérer la requête HTTP. N'oubliez pas de vérifier le statut de la réponse pour confirmer que la page a été téléchargée avec succès.

In [15]:
import requests

# URL of the Wikipedia page
url = "https://fr.wikipedia.org/wiki/Paris"

# Send a GET request to the URL
response = requests.get(url)

# Save the content as an HTML file
with open("paris.html", "w", encoding='utf-8') as file:
    file.write(response.text)

print("HTML page downloaded and saved as paris.html")


HTML page downloaded and saved as paris.html


##### Étape 2 : Charger le Contenu HTML
Chargez le fichier HTML téléchargé pour une analyse plus approfondie.
- **Commentaire** : Parser le HTML est crucial pour extraire des données. Assurez-vous d'utiliser une bibliothèque comme BeautifulSoup qui peut naviguer efficacement dans la structure HTML.

Familiarisez-vous avec les méthodes de `BeautifulSoup` pour trouver des éléments dans le HTML, telles que `find()` et `find_all()`.

In [16]:
%pip install beautifulsoup4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
from bs4 import BeautifulSoup

# Load the HTML file
with open("paris.html", "r", encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
print("HTML content loaded.")


HTML content loaded.


##### Étape 3 : Extraire et Analyser les Mots
Comptez les occurrences de chaque mot sur la page.
- **Commentaire** : Envisagez de normaliser le texte en le convertissant en minuscules pour éviter de compter le même mot en différentes majuscules séparément. Nous utilisons des expressions régulières pour filtrer efficacement les caractères non alphanumériques lors de la séparation du texte en mots.

In [19]:
from collections import Counter
import re

# Extract text from the HTML content
text = soup.get_text()

# Clean and split text into words
words = re.findall(r'\w+', text.lower())
word_count = Counter(words)

# Display the 10 most common words
print(word_count.most_common(5))


[('de', 3538), ('la', 2128), ('le', 1518), ('et', 1326), ('paris', 1214)]


##### Étape 4 : Extraire les Liens
Identifiez tous les liens internes et externes de la page.

- **Commentaire** : Comprendre la différence entre les liens internes et externes est important pour la catégorisation.
- **Indice** : Vérifiez l'attribut `href` des balises d'ancrage (`<a>`) pour déterminer le type de lien.

In [20]:
# extraction des lien <a> du html
links = soup.find_all('a')
intern_links = []
extern_links = []

# Extract internal and external links
for link in links:
    href = link.get('href')
    if href:
        if href.startswith('http'):
            extern_links.append(href)
        else:
            intern_links.append(href)


##### Étape 5 : Extraire les Images et Leurs Tailles
Identifiez toutes les images sur la page et obtenez leurs tailles.

- **Commentaire** : Soyez conscient que les images ne sont pas toujours stockées dans le même format. Assurez-vous de construire les bonnes URLs pour elles.
- **Indice** : Vous devrez peut-être vérifier les attributs des balises `<img>` pour obtenir des informations supplémentaires, telles que la taille des images si disponible.

In [23]:
# ectract all images
images = soup.find_all('img')

# Extract image URLs and size id possible
image_data = []
for image in images:
    src = image.get('src')
    alt = image.get('alt')
    width = image.get('width')
    height = image.get('height')
    image_data.append({'src': src, 'alt': alt, 'width': width, 'height': height})
image_data[:2]

[{'src': '/static/images/icons/wikipedia.png',
  'alt': '',
  'width': '50',
  'height': '50'},
 {'src': '/static/images/mobile/copyright/wikipedia-wordmark-fr.svg',
  'alt': 'Wikipédia',
  'width': None,
  'height': None}]

##### Étape 6 : Extraire les Nombres, Dates et Coordonnées Géographiques
Identifiez les nombres, dates et coordonnées géographiques dans le texte.

- **Commentaire** : Différents formats pour les dates et les nombres peuvent compliquer l'extraction. Considérez les diverses manières dont ces éléments peuvent apparaître sur la page.
- **Indice** : Utilisez des expressions régulières adaptées à des motifs spécifiques (par exemple, formats de date ou coordonnées géographiques) pour les identifier avec précision.

In [25]:
# extract dates with regex, and geographical data, and numbers
dates = re.findall(r'\d{1,2} [a-zA-Z]+ \d{4}', text)
geographical_data = re.findall(r'\d{2}\.\d{4,6}, \d{2}\.\d{4,6}', text)
numbers = re.findall(r'\d+', text)
geographical_data

[]

##### Étape 7 : Identifier les Noms Propres
Extraire les noms propres du texte.

- **Commentaire** : Les noms propres peuvent inclure des noms de personnes, de lieux et d'organisations. Les identifier correctement peut améliorer votre analyse des données.
- **Indice** : Utilisez des techniques de traitement du langage naturel (NLP), telles que la reconnaissance d'entités nommées, pour automatiser l'identification des noms propres.

In [27]:
%pip install spacy

   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ------------------------- -------------- 7.6/11.8 MB 39.3 MB/s eta 0:00:01
   ---------------------------------------- 11.8/11.8 MB 36.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 53.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 37.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ------------------------ --------------- 9.4/15.6 MB 48.8 MB/s eta 0:00:01
   ---------------------------------------- 15.6/15.6 MB 40.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.4 MB ? eta -:--:--
   ---------------------------------------- 6.4/6.4 MB 39.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.4 MB ? eta -:--:--
   ------------------------------------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.0.2 which is incompatible.


In [30]:
!python -m spacy download fr_core_news_md

     ---------------------------------------- 0.0/45.8 MB ? eta -:--:--
     ----- ---------------------------------- 6.6/45.8 MB 40.3 MB/s eta 0:00:01
     ------------- ------------------------- 15.7/45.8 MB 41.2 MB/s eta 0:00:01
     -------------------- ------------------ 24.1/45.8 MB 41.3 MB/s eta 0:00:01
     ---------------------------- ---------- 33.3/45.8 MB 42.3 MB/s eta 0:00:01
     ----------------------------------- --- 42.2/45.8 MB 42.6 MB/s eta 0:00:01
     --------------------------------------- 45.8/45.8 MB 38.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [35]:
# Extract all noms propres with NLP
import spacy

nlp = spacy.load("fr_core_news_md")
doc = nlp(text)
named_entities = [(ent.text, ent.label_) for ent in doc.ents]
named_entities[:5]
# get all labels
labels = set([ent.label_ for ent in doc.ents])
labels

{'LOC', 'MISC', 'ORG', 'PER'}

In [37]:
# get all PERS
persons = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == 'PER']
persons

[('Débuter', 'PER'),
 ('WikipédiaAideCommunautéModifications', 'PER'),
 ('Apparence', 'PER'),
 ('Urbanisme', 'PER'),
 ('Presse', 'PER'),
 ('BaliBoarischŽemaitėškaBikol CentralБеларускаяБеларуская', 'PER'),
 ('Gõychi Konknni𐌲𐌿𐍄𐌹𐍃𐌺ગુજરાતીGungbeGaelgHausa客家語', 'PER'),
 ('Hak-kâ-ngîHawaiʻiעבריתहिन्दीFiji', 'PER'),
 ('JawaქართულიQaraqalpaqshaTaqbaylitАдыгэбзэKabɩyɛTyapKongoKumoringҚазақшаKalaallisutភាសាខ្មែរಕನ್ನಡ한국어Перем комиКъарачай',
  'PER'),
 ('Franca NovaLugandaLimburgsLigureLadinLombardLingálaລາວLietuviųLatgaļuLatviešuमैथिलीМокшеньMalagasyОлык',
  'PER'),
 ('Général', 'PER'),
 ('Anne Hidalgo', 'PER'),
 ('Clovis', 'PER'),
 ('baron Haussmann', 'PER'),
 ('Roland Garros', 'PER'),
 ('Thiers', 'PER'),
 ('Thiers', 'PER'),
 ('baron Haussmann', 'PER'),
 ('ville[2].', 'PER'),
 ('Jean-Paul II', 'PER'),
 ('Richard-Lenoir', 'PER'),
 ('Lamartine', 'PER'),
 ('Javel', 'PER'),
 ('Saint-Gervais', 'PER'),
 ('Charles V', 'PER'),
 ('Saint-Martin', 'PER'),
 ('Miocène', 'PER'),
 ('Pliocène', 'PER'),
 ('pier

##### Étape 8 : Extraire des Données Structurées (Tableaux)
Identifiez et extrayez des données des tableaux présents dans le HTML.

- **Commentaire** : Les tableaux contiennent souvent des données organisées qui peuvent être utiles pour l'analyse. Assurez-vous de capturer à la fois les cellules d'en-tête et les cellules de données.
- **Indice** : Familiarisez-vous avec la structure des tableaux HTML, y compris comment naviguer dans les lignes (`<tr>`) et les cellules (`<td>` et `<th>`).

In [40]:
# extract tables
tables = soup.find_all('table')
table_data = []
for table in tables:
    rows = table.find_all('tr')
    r = []
    for row in rows:
        cells = row.find_all(['th', 'td'])
        r.append([cell.text for cell in cells])
    table_data.append(r)
table_data

[[['Paris\n'],
  ["\nDe haut en bas et de gauche à droite\xa0: Vue sur la Seine et la tour Eiffel, le pont des Arts avec au loin la cathédrale Notre-Dame, l'opéra Garnier, l'Arc de Triomphe et enfin le palais du Louvre avec sa pyramide."],
  ['Blason\n', 'Logo\n'],
  ['Administration'],
  ['Pays\n', ' France\n'],
  ['Région\n', 'Île-de-France (préfecture)\n'],
  ['Arrondissement\n', 'Chef-lieu de vingt arrondissements\n'],
  ['Intercommunalité\n', 'Métropole du Grand Paris (siège)\n'],
  ['Maire Mandat\n', 'Anne Hidalgo (PS) 2020–2026\n'],
  ['Code postal\n', 'Selon l’arrondissement, de 75001 à 75020 et 75116\n'],
  ['Code commune\n',
   '75056 Codes des arrondissements\xa0: de 75101 à 75120\n'],
  ['Démographie'],
  ['Gentilé\n', 'Parisien, Parisienne\n'],
  ['Populationmunicipale\n', '2\xa0133\xa0111\xa0hab. (2021 )\n'],
  ['Densité', '20\xa0238\xa0hab./km2'],
  ['Population agglomération\n', '10\xa0890\xa0751\xa0hab. (2021)\n'],
  ['Géographie'],
  ['Coordonnées ', '48°\xa051′\xa024

##### Étape 9 : Différencier les Sections et les Paragraphes
Identifiez et séparez les sections et les paragraphes dans le contenu.

- **Commentaire** : Les sections aident à comprendre l'organisation du contenu. Reconnaître les différents niveaux de titres peut faciliter la navigation dans le contenu.
- **Indice** : Utilisez les balises appropriées (`<h1>`, `<h2>`, etc.) pour différencier les sections et assurez-vous de capturer leur contenu associé, comme les paragraphes.

In [57]:
# extract all headers with there paragraphes
headers = [ h.text for h in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
paragraphs = [p.text for p in soup.find_all('p')]

headers[:2]

['Sommaire', 'Paris']

## Exercice 1.5 [★★★]
Analysez le texte de la page Wikipedia téléchargée en appliquant le stemming, l'extraction d'n-grammes, l'étiquetage des parties du discours (PoS), la lemmatisation, l'analyse morphologique et la reconnaissance d'entités nommées. Comparez les résultats de NLTK et de spaCy pour évaluer leur efficacité dans les tâches d'analyse de texte.

### Prérequis

Assurez-vous d'avoir les bibliothèques nécessaires installées. Vous pouvez les installer en utilisant pip si ce n'est pas déjà fait :

In [58]:
%pip install spacy gensim wordcloud seaborn

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ---------------------------------------- 0.0/56.0 MB ? eta -:--:--
     ---- ----------------------------------- 5.8/56.0 MB 39.3 MB/s eta 0:00:02
     ------- ------------------------------- 10.5/56.0 MB 28.5 MB/s eta 0:00:02
     ----------- --------------------------- 16.8/56.0 MB 27.9 MB/s eta 0:00:02
     ---------------- ---------------------- 23.9/56.0 MB 29.6 MB/s eta 0:00:02
     

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + meson setup C:\Users\CPE\AppData\Local\Temp\pip-install-2cwgn85o\scipy_17efc76a35de4743bc2c6d3d47d783dc C:\Users\CPE\AppData\Local\Temp\pip-install-2cwgn85o\scipy_17efc76a35de4743bc2c6d3d47d783dc\.mesonpy-7ki4nira\build -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\CPE\AppData\Local\Temp\pip-install-2cwgn85o\scipy_17efc76a35de4743bc2c6d3d47d783dc\.mesonpy-7ki4nira\build\meson-python-native-file.ini
      The Meson build system
      Version: 1.6.0
      Source dir: C:\Users\CPE\AppData\Local\Temp\pip-install-2cwgn85o\scipy_17efc76a35de4743bc2c6d3d47d783dc
      Build dir: C:\Users\CPE\AppData\Local\Temp\pip-install-2cwgn85o\scipy_17efc76a35de4743bc2c6d3d47d783dc\.mesonpy-7ki4nira\build
      Build type: native build
      Project name: SciPy
      Project version: 1.11.1
      
      ..\..\meson.bu

In [2]:
%pip install ntlk

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [3]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CPE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\CPE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\CPE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\CPE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CPE\AppData\Roaming\nltk_data...
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\CPE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C

True

#### Étape 1 : Charger la Page Wikipedia
Commencez par charger le fichier HTML que vous avez enregistré précédemment et extraire le texte.

In [4]:
from bs4 import BeautifulSoup

# Load the HTML file
with open("paris.html", "r", encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, "html.parser")
text = soup.get_text()

#### Étape 2 : Appliquer des Algorithmes de Stemming
Utilisez les stemmers Porter et Snowball de NLTK pour réduire les mots du texte à leur racine.

In [5]:
import nltk
from nltk.stem import PorterStemmer, SnowballStemmer
from collections import Counter
import re

# Tokenize and clean the text
words = re.findall(r'\w+', text.lower())

# Initialize stemmers
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer("english")

# Apply stemming
porter_stems = [porter_stemmer.stem(word) for word in words]
snowball_stems = [snowball_stemmer.stem(word) for word in words]

# Count unique stems
porter_stem_count = Counter(porter_stems)
snowball_stem_count = Counter(snowball_stems)

# Display the most common stems and count of unique stems
print("Most common Porter stems:", porter_stem_count.most_common(10))
print("Unique Porter stems count:", len(porter_stem_count))

print("Most common Snowball stems:", snowball_stem_count.most_common(10))
print("Unique Snowball stems count:", len(snowball_stem_count))

Most common Porter stems: [('de', 4484), ('le', 2348), ('la', 2129), ('et', 1326), ('pari', 1215), ('l', 1045), ('en', 924), ('à', 923), ('du', 781), ('d', 754)]
Unique Porter stems count: 8006
Most common Snowball stems: [('de', 3538), ('la', 2128), ('le', 1518), ('et', 1326), ('pari', 1215), ('l', 1045), ('des', 946), ('en', 924), ('à', 923), ('les', 830)]
Unique Snowball stems count: 8079


#### Étape 3 : Extraire des N-grammes
Générez et affichez les n-grammes les plus courants (1-grammes à 5-grammes) du texte.

In [11]:
# extract N-grams from 1 to 5
from nltk.util import ngrams

# Extract n-grams from 1 to 5
n_grams = []
for n in range(1, 6):
    n_grams.extend(ngrams(words, n))
    
# Count unique n-grams
n_gram_count = Counter(n_grams)
n_gram_count.most_common(5)

[(('de',), 3538),
 (('la',), 2128),
 (('le',), 1518),
 (('et',), 1326),
 (('paris',), 1214)]

#### Étape 4 : Étiquetage des Parties du Discours (PoS)
Utilisez NLTK ou spaCy pour effectuer l'étiquetage des parties du discours sur le texte.

In [12]:
# extract POS
from nltk import pos_tag
from nltk.tokenize import word_tokenize

# Tokenize the text
tokens = word_tokenize(text)

# Extract POS tags
pos_tags = pos_tag(tokens)
pos_tags[:5]

[('Paris', 'NNP'),
 ('—', 'NNP'),
 ('Wikipédia', 'NNP'),
 ('Aller', 'NNP'),
 ('au', 'NN')]

#### Étape 5 : Lemmatisation
Appliquez la lemmatisation en utilisant NLTK ou spaCy.

In [13]:
# Lemmatisation

from nltk.stem import WordNetLemmatizer

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize the words
lemmas = [lemmatizer.lemmatize(word) for word in words]

# Count unique lemmas
lemma_count = Counter(lemmas)

# Display the most common lemmas and count of unique lemmas
print("Most common lemmas:", lemma_count.most_common(10))


Most common lemmas: [('de', 4484), ('le', 2348), ('la', 2129), ('et', 1326), ('paris', 1214), ('l', 1045), ('en', 924), ('à', 923), ('du', 781), ('d', 754)]


#### Étape 6 : Analyse Morphologique
Utilisez spaCy pour effectuer une analyse morphologique sur le texte.

In [17]:
# Morphological analysis
import spacy

nlp = spacy.load("fr_core_news_md")
doc = nlp(text)

# Extract morphological analysis
morphological_analysis = [(token.text, token.morph) for token in doc]
morphological_analysis[:5]
    

[('\n\n\n\n', ),
 ('Paris', Gender=Masc|Number=Sing),
 ('—', ),
 ('Wikipédia', ),
 ('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
  )]

#### Étape 7 : Reconnaissance d'Entités Nommées (NER)
Utilisez spaCy pour identifier les entités nommées dans le texte.

In [19]:
# get entities from doc
entities = [(ent.text, ent.label_) for ent in doc.ents]
entities[:5]

[('Paris', 'LOC'),
 ('Wikipédia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller',
  'MISC'),
 ('Menu', 'MISC'),
 ('Navigation\n\t\n\n\n', 'MISC'),
 ('AccueilPortails', 'ORG')]

#### Étape 8 : Distribution de Fréquence des Mots
Visualisez la distribution de fréquence des mots à l'aide de Matplotlib.

In [23]:
# show word frequencies
import matplotlib.pyplot as plt

# Get the most common words and their frequencies


# Plot the word frequencies
plt.figure(figsize=(10, 6))
plt.bar(lemma)
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('10 Most Common Words')
plt.show()



NameError: name 'lemma' is not defined

<Figure size 1000x600 with 0 Axes>

#### Étape 9 : Créer un Nuage de Mots

Générez un nuage de mots pour visualiser les mots les plus fréquents.

#### Étape 10 : Visualisation des Entités Nommées

Visualisez les entités nommées reconnues dans le texte à l'aide de Matplotlib.

#### Étape 11 : Visualisation des Noms les Plus Courants

Visualisez les noms les plus courants dans le texte, ce qui peut fournir des informations sur les principaux sujets abordés.